In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import numpy as np
# sns.set_theme()
# palette = sns.color_palette("viridis", as_cmap=True)
# palette = sns.color_palette("hsv", as_cmap=True)
# palette = sns.color_palette("Spectral", as_cmap=True)

palette = "viridis"

- Download and compile [HypoDD](https://www.ldeo.columbia.edu/~felixw/hypoDD.html)
- Download test files from [Zhu et al. (2021)](https://arxiv.org/abs/2109.09008)
```bash
curl -O -J -L https://osf.io/aw53b/download
curl -O -J -L https://osf.io/y879e/download
```
- Convert GaMMA catalog and run HypoDD relocation
```
python gamma2hypodd.py
```

In [ ]:
region_name = "Ridgecrest"
xlim = (-117.8, -117.3)
ylim = (35.5, 36.0)
zlim = (0, 15)
zlim_special = (0, 20)
size = 1.0
alpha = 0.5
max_sigma = 0.6

# region_name = "PuertoRico"
# # xlim = (-68, -65)
# # ylim = (17, 19)
# xlim = (-67.2, -66.6)
# ylim = (17.75, 18.1)
# zlim = (0, 25)
# zlim_special = None
# size = 0.5
# alpha = 0.3
# max_sigma = 0.5

# region_name = "Hawaii"
# xlim = (-156.00, -154.75)
# ylim = (18.9, 19.9)
# zlim = (0, 40)
# zlim_special = None
# size = 1.0
# alpha = 0.5
# max_sigma = 1.5


In [ ]:

# catalog_hypoinverse = pd.read_csv("catOut.sum", sep="\s+")
catalog_hypoDD = pd.read_csv(f"./{region_name}/hypoDD_catalog.txt", sep="\s+", names=["ID", "LAT", "LON", "DEPTH", "X", "Y", "Z", "EX", "EY", "EZ", "YR", "MO", "DY", "HR", "MI", "SC", "MAG", "NCCP", "NCCS", "NCTP",
"NCTS", "RCC", "RCT", "CID"])
catalog_hypoDD["time"] = catalog_hypoDD.apply(lambda x: f'{x["YR"]:04.0f}-{x["MO"]:02.0f}-{x["DY"]:02.0f}T{x["HR"]:02.0f}:{x["MI"]:02.0f}:{min(x["SC"], 59.999):05.3f}', axis=1)
catalog_hypoDD["time"] = catalog_hypoDD["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f"))

catalog_gamma = pd.read_csv(f"./{region_name}/gamma_catalog.csv", sep="\t")
catalog_gamma["time"] = catalog_gamma["time"].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%f"))

In [ ]:
catalog_hypoDD["latitude"] = catalog_hypoDD["LAT"]
catalog_hypoDD["longitude"] = catalog_hypoDD["LON"]
catalog_hypoDD["depth(m)"] = catalog_hypoDD["DEPTH"] * 1e3
catalog_hypoDD["magnitude"] = catalog_hypoDD["MAG"] 

In [ ]:
picks_gamma = pd.read_csv(f"./{region_name}/gamma_picks.csv", sep="\t")
print(f"Number of stations: {len(set(picks_gamma['id']))}")

In [ ]:
catalog_gamma["sigma"] = catalog_gamma["covariance"].apply(lambda x: float(x.split(",")[0]))
catalog_gamma_selected = catalog_gamma[catalog_gamma["sigma"] < max_sigma]

In [ ]:
c_gamma = (np.array(catalog_gamma_selected["time"]) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
c_hypodd = (np.array(catalog_hypoDD["time"]) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
t_gamma = catalog_gamma_selected["time"]
t_hypodd = catalog_hypoDD["time"]

In [ ]:
import pygmt
region = xlim + ylim
# region = np.array(region) + np.array([-1, 1, -1, 1])*0.2
stations = pd.read_csv("stations.csv", sep="\t")
fig = pygmt.Figure()
fig.basemap(region=region, projection="M6i", frame=True)
fig.grdimage("@earth_relief_15s", cmap="topo", shading=True)
# fig.grdimage("@earth_relief_03s", cmap="sealand", shading=True)
# fig.colorbar()

# grid = pygmt.datasets.load_earth_relief(resolution="15s", region=region)
# dgrid = pygmt.grdgradient(grid=grid, radiance=[270, 30])
# pygmt.makecpt(cmap="gray", series=[-50000, 20000, 1000], continuous=True)
# fig.grdimage(grid=grid, cmap=True, shading=True)

# fig.grdimage(grid=dgrid, cmap=True, shading=True)
# fig.colorbar(truncate=[-4000, 4001])

# fig.plot(x=stations["longitude"], y=stations["latitude"], style="t0.5", color="blue", pen="black", label="Station")
# fig.plot(x=catalog_hypoDD["longitude"], y=catalog_hypoDD["latitude"], style="c", size=1, color="black")

fig.savefig(f"{region_name}/topography.pdf")
fig.savefig(f"{region_name}/topography.png")
fig.show()

In [ ]:
dgrid.max(), dgrid.min()

In [ ]:
plt.figure(figsize=(15, 6))

grid = pygmt.datasets.load_earth_relief(resolution="03s", region=region)
dgrid = pygmt.grdgradient(grid=grid, radiance=[135, 25])
# dgrid = pygmt.grdgradient(grid=grid)

xgrid = np.linspace(xlim[0], xlim[1], grid.shape[1])
ygrid = np.linspace(ylim[0], ylim[1], grid.shape[0])

im_ratio = (ylim[1]-ylim[0])/(xlim[1]-xlim[0])

plt.subplot(121)
plt.pcolormesh(xgrid, ygrid, dgrid, shading="gouraud", cmap="gray", alpha=0.3, vmin=-1.5, rasterized=True)
im = plt.scatter(catalog_gamma_selected["longitude"], catalog_gamma_selected["latitude"], s=size, c=c_gamma, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
# plt.pcolormesh(xgrid, ygrid, dgrid, shading="gouraud", cmap="gray", alpha=0.1, rasterized=True)
plt.title(f"GaMMA ($\sigma$ < {max_sigma:.1f}s): {len(catalog_gamma_selected)}")
plt.axis("scaled")
plt.xlim(xlim)
plt.ylim(ylim)
plt.xlabel("Latitude")
plt.ylabel("Longitude")

cbar = plt.colorbar(im, fraction=0.047*im_ratio)
cbar.set_ticks(np.linspace(c_gamma.min(), c_gamma.max(), 4))
cbar.ax.set_yticklabels([pd.to_datetime(x, unit='s').strftime('%b %d %Y') for x in np.linspace(t_gamma.min().timestamp(), t_gamma.max().timestamp(), 4)])


plt.subplot(122)
plt.pcolormesh(xgrid, ygrid, dgrid, shading="gouraud", cmap="gray", alpha=0.3, vmin=-1.5, rasterized=True)
im = plt.scatter(catalog_hypoDD["LON"], catalog_hypoDD["LAT"], s=size, c=c_hypodd, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
# plt.pcolormesh(xgrid, ygrid, dgrid, shading="gouraud", cmap="gray", alpha=0.1, rasterized=True)
plt.title(f"HypoDD: {len(catalog_hypoDD)}")
plt.axis("scaled")
plt.xlim(xlim)
plt.ylim(ylim)
plt.xlabel("Latitude")
plt.ylabel("Longitude")

cbar = plt.colorbar(im, fraction=0.047*im_ratio)
cbar.set_ticks(np.linspace(c_hypodd.min(), c_hypodd.max(), 4))
cbar.ax.set_yticklabels([pd.to_datetime(x, unit='s').strftime('%b %d %Y') for x in np.linspace(t_hypodd.min().timestamp(), t_hypodd.max().timestamp(), 4)])

plt.tight_layout()

plt.savefig(f"{region_name}/GaMMA2HypoDD_latitude_vs_longitude_color_by_time.pdf", bbox_inches="tight", dpi=600)
plt.savefig(f"{region_name}/GaMMA2HypoDD_latitude_vs_longitude_color_by_time.png", bbox_inches="tight", dpi=600)
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

im_ratio = (ylim[1]-ylim[0])/(xlim[1]-xlim[0])

c = catalog_gamma_selected["depth(m)"].copy()/1e3
c[c<zlim[0]] = zlim[0]
if zlim_special is None:
    c[c>zlim[1]] = zlim[1]
else:
    c[c>zlim_special[1]] = zlim_special[1]

plt.subplot(121)
plt.pcolormesh(xgrid, ygrid, dgrid, shading="gouraud", cmap="gray", alpha=0.3, vmin=-1.5, rasterized=True)
plt.scatter(catalog_gamma_selected["longitude"], catalog_gamma_selected["latitude"], s=size, c=c, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"GaMMA ($\sigma$ < {max_sigma:.1f}s): {len(catalog_gamma_selected)}")
plt.axis("scaled")
plt.xlim(xlim)
plt.ylim(ylim)
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.colorbar(label="Depth (km)", fraction=0.047*im_ratio)

c = catalog_hypoDD["DEPTH"].copy()
c[c<zlim[0]] = zlim[0]
c[c>zlim[1]] = zlim[1]

plt.subplot(122)
plt.pcolormesh(xgrid, ygrid, dgrid, shading="gouraud", cmap="gray", alpha=0.3, vmin=-1.5, rasterized=True)
im = plt.scatter(catalog_hypoDD["LON"], catalog_hypoDD["LAT"], s=size, c=c, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"HypoDD: {len(catalog_hypoDD)}")
plt.axis("scaled")
plt.xlim(xlim)
plt.ylim(ylim)
plt.xlabel("Latitude")
plt.ylabel("Longitude")
plt.colorbar(label="Depth (km)", fraction=0.047*im_ratio)

plt.tight_layout()

plt.savefig(f"{region_name}/GaMMA2HypoDD_latitude_vs_longitude_color_by_depth.pdf", bbox_inches="tight", dpi=600)
plt.savefig(f"{region_name}/GaMMA2HypoDD_latitude_vs_longitude_color_by_depth.png", bbox_inches="tight", dpi=600)
plt.show()

In [ ]:
sns.set_theme()

In [ ]:
# plt.figure(figsize=(15, 6))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))

c = catalog_gamma_selected["latitude"].copy()
c[c < ylim[0]] = ylim[0]
c[c > ylim[1]] = ylim[1]

ax = axes[0]
im = ax.scatter(catalog_gamma_selected["longitude"], catalog_gamma_selected["depth(m)"]/1e3, s=size, c=c, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
ax.set_title(f"GaMMA ($\sigma$ < {max_sigma:.1f}s): {len(catalog_gamma_selected)}")
ax.set_xlim(xlim)
if zlim_special is None:
    ax.set_ylim(zlim)
else:
    ax.set_ylim(zlim_special)
ax.set_xlabel("Longitude")
ax.set_ylabel("Depth (km)")
ax.invert_yaxis()
fig.colorbar(im, ax=ax, label="Latitude")

c = catalog_hypoDD["LAT"].copy()
c[c < ylim[0]] = ylim[0]
c[c > ylim[1]] = ylim[1]

ax = axes[1]
im = ax.scatter(catalog_hypoDD["LON"], catalog_hypoDD["DEPTH"], s=size, c=c, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
ax.set_title(f"HypoDD: {len(catalog_hypoDD)}")
ax.set_xlim(xlim)
ax.set_ylim(zlim)
ax.set_xlabel("Longitude")
ax.set_ylabel("Depth (km)")
ax.invert_yaxis()
fig.colorbar(im, ax=ax, label="Latitude")

# fig.colorbar(im, ax=axes.ravel().tolist(), label="Latitude")

plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_longitude_color_by_latitude.pdf", bbox_inches="tight", dpi=600)
plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_longitude_color_by_latitude.png", bbox_inches="tight", dpi=600)
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

c = catalog_gamma_selected["longitude"].copy()
c[c < xlim[0]] = xlim[0]
c[c > xlim[1]] = xlim[1]
plt.subplot(121)
plt.scatter(catalog_gamma_selected["latitude"], catalog_gamma_selected["depth(m)"]/1e3, c=c, s=size, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"GaMMA ($\sigma$ < {max_sigma:.1f}s): {len(catalog_gamma_selected)}")
plt.xlim(ylim)
if zlim_special is None:
    plt.ylim(zlim)
else:
    plt.ylim(zlim_special)
plt.gca().invert_yaxis()
plt.xlabel("Latitude")
plt.ylabel("Depth (km)")
plt.colorbar(label="Longitude")

c = catalog_hypoDD["LON"].copy()
c[c < xlim[0]] = xlim[0]
c[c > xlim[1]] = xlim[1]
plt.subplot(122)
plt.scatter(catalog_hypoDD["LAT"], catalog_hypoDD["DEPTH"], s=size, c=c, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"HypoDD: {len(catalog_hypoDD)}")
plt.xlim(ylim)
plt.ylim(zlim)
plt.gca().invert_yaxis()
plt.xlabel("Latitude")
plt.ylabel("Depth (km)")
plt.colorbar(label="Longitude")

plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_latitude_color_by_longitude.pdf", bbox_inches="tight", dpi=600)
plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_latitude_color_by_longitude.png", bbox_inches="tight", dpi=600)
plt.show()

In [ ]:
import matplotlib.dates as mdates
plt.figure(figsize=(15, 6))

# t_gamma = mdates.epoch2num(t_gamma)
# t_hypodd = mdates.epoch2num(t_hypodd)
# t_gamma = mdates.datetime(t_gamma)

c = catalog_gamma_selected["latitude"].copy()
c[c < ylim[0]] = ylim[0]
c[c > ylim[1]] = ylim[1]

plt.subplot(121)
plt.scatter(catalog_gamma_selected["longitude"], catalog_gamma_selected["depth(m)"]/1e3, s=size, c=c_gamma, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"GaMMA ($\sigma$ < {max_sigma:.1f}s): {len(catalog_gamma_selected)}")
plt.xlim(xlim)
if zlim_special is None:
    plt.ylim(zlim)
else:
    plt.ylim(zlim_special)
plt.gca().invert_yaxis()
plt.xlabel("Longitude")
plt.ylabel("Depth (km)")

cbar = plt.colorbar()
cbar.set_ticks(np.linspace(c_gamma.min(), c_gamma.max(), 4))
cbar.ax.set_yticklabels([pd.to_datetime(x, unit='s').strftime('%b %d %Y') for x in np.linspace(t_gamma.min().timestamp(), t_gamma.max().timestamp(), 4)])


c = catalog_hypoDD["LAT"].copy()
c[c < ylim[0]] = ylim[0]
c[c > ylim[1]] = ylim[1]
plt.subplot(122)
plt.scatter(catalog_hypoDD["LON"], catalog_hypoDD["DEPTH"], s=size, c=c_hypodd, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"HypoDD: {len(catalog_hypoDD)}")
plt.xlim(xlim)
plt.ylim(zlim)
plt.gca().invert_yaxis()
plt.xlabel("Longitude")
plt.ylabel("Depth (km)")

cbar = plt.colorbar()
cbar.set_ticks(np.linspace(c_hypodd.min(), c_hypodd.max(), 4))
cbar.ax.set_yticklabels([pd.to_datetime(x, unit='s').strftime('%b %d %Y') for x in np.linspace(t_hypodd.min().timestamp(), t_hypodd.max().timestamp(), 4)])

plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_longitude_color_by_time.pdf", bbox_inches="tight", dpi=600)
plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_longitude_color_by_time.png", bbox_inches="tight", dpi=600)
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))

c = catalog_gamma_selected["longitude"].copy()
c[c < xlim[0]] = xlim[0]
c[c > xlim[1]] = xlim[1]
plt.subplot(121)
plt.scatter(catalog_gamma_selected["latitude"], catalog_gamma_selected["depth(m)"]/1e3, c=c_gamma, s=size, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"GaMMA ($\sigma$ < {max_sigma:.1f}s): {len(catalog_gamma_selected)}")
plt.xlim(ylim)
if zlim_special is None:
    plt.ylim(zlim)
else:
    plt.ylim(zlim_special)
plt.gca().invert_yaxis()

cbar = plt.colorbar()
cbar.set_ticks(np.linspace(c_gamma.min(), c_gamma.max(), 4))
cbar.ax.set_yticklabels([pd.to_datetime(x, unit='s').strftime('%b %d %Y') for x in np.linspace(t_gamma.min().timestamp(), t_gamma.max().timestamp(), 4)])


c = catalog_hypoDD["LON"].copy()
c[c < xlim[0]] = xlim[0]
c[c > xlim[1]] = xlim[1]
plt.subplot(122)
plt.scatter(catalog_hypoDD["LAT"], catalog_hypoDD["DEPTH"], s=size, c=c_hypodd, alpha=alpha, marker=",", cmap=palette, linewidth=0, rasterized=True)
plt.title(f"HypoDD: {len(catalog_hypoDD)}")
plt.xlim(ylim)
plt.ylim(zlim)
plt.gca().invert_yaxis()

cbar = plt.colorbar()
cbar.set_ticks(np.linspace(c_hypodd.min(), c_hypodd.max(), 4))
cbar.ax.set_yticklabels([pd.to_datetime(x, unit='s').strftime('%b %d %Y') for x in np.linspace(t_hypodd.min().timestamp(), t_hypodd.max().timestamp(), 4)])

plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_latitude_color_by_time.pdf", bbox_inches="tight", dpi=600)
plt.savefig(f"{region_name}/GaMMA2HypoDD_depth_vs_latitude_color_by_time.png", bbox_inches="tight", dpi=600)
plt.show()